In [13]:
import tensorflow as tf
import wave, os, glob
from IPython.display import display, Audio
from scipy.io.wavfile import read, write
import numpy as np
import pickle as pkl
import itertools

from utils import *

# for auto-reloading external modules
# see http://stackoverflow.com/questions/1907993/autoreload-of-modules-in-ipython
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Creating a dataset to train on
Add more paths to `data_paths` to increase number of words

In [23]:
PATH_DATA = './test_data/'
data_paths = ['water_40.wav', 'greasy_2.wav', 'suit_16.wav', 'box_3.wav', 'under_16.wav']
word_dict = {}
for path in data_paths:
    word = normalize(np.array(read(PATH_DATA + path)[1]))
    word = word / np.max(np.abs(word))
    word_dict[path[:path.find('_')]] = word
    
for key in word_dict.keys():
    print(key)
    play(word_dict[key])

water


greasy


suit


box


under


In [24]:
all_combinations = set()

# Generates all possible combinations of up to k words (and their permutations)
k = 2
for i in range(1, k+1):
    for perm in itertools.permutations(word_dict.keys(), i):
        all_combinations.add(perm)
    
# Specify which combinations of words should be withheld from training data
withheld_combinations = set([
    ('under', 'water'),
    ('water', 'under')
])

training_combinations = all_combinations - withheld_combinations

print("training combinations: \n", training_combinations, len(training_combinations))
print("withheld combinations: \n", withheld_combinations, len(withheld_combinations))

training combinations: 
 {('under',), ('greasy',), ('box', 'under'), ('box', 'greasy'), ('greasy', 'suit'), ('water', 'greasy'), ('suit',), ('greasy', 'box'), ('suit', 'under'), ('box',), ('suit', 'greasy'), ('box', 'suit'), ('greasy', 'water'), ('water', 'suit'), ('water',), ('water', 'box'), ('suit', 'box'), ('box', 'water'), ('under', 'greasy'), ('suit', 'water'), ('greasy', 'under'), ('under', 'suit'), ('under', 'box')} 23
withheld combinations: 
 {('water', 'under'), ('under', 'water')} 2


In [34]:
sample_size = 100
desired_length = 16384 * 2

train_dict = {}
for perm in training_combinations:
    train_dict[perm] = generate_data([word_dict[w] for w in perm], desired_length, sample_size)
    
print(train_dict.keys())

dict_keys([('under',), ('greasy',), ('box', 'under'), ('box', 'greasy'), ('greasy', 'suit'), ('water', 'greasy'), ('suit',), ('greasy', 'box'), ('suit', 'under'), ('box',), ('suit', 'greasy'), ('box', 'suit'), ('greasy', 'water'), ('water', 'suit'), ('water',), ('water', 'box'), ('suit', 'box'), ('box', 'water'), ('under', 'greasy'), ('suit', 'water'), ('greasy', 'under'), ('under', 'suit'), ('under', 'box')])


In [37]:
path = './generated_data/'
amplitude = np.iinfo(np.int16).max
for key in train_dict.keys():
    save_name = ''
    for w in key:
        save_name += w + '_'
    print(save_name)
    for i, audio in enumerate(train_dict[key]):
        filename = path + save_name + str(i) + '.wav'
        write(filename, 16000, (audio * amplitude).astype(np.int16))

under_
greasy_
box_under_
box_greasy_
greasy_suit_
water_greasy_
suit_
greasy_box_
suit_under_
box_
suit_greasy_
box_suit_
greasy_water_
water_suit_
water_
water_box_
suit_box_
box_water_
under_greasy_
suit_water_
greasy_under_
under_suit_
under_box_
